In [2]:
import sys
!{sys.executable} -m pip install pafy
!{sys.executable} -m pip install youtube-dl
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install facenet-pytorch

    100% |################################| 1.9MB 273kB/s eta 0:00:01
    100% |################################| 1.9MB 271kB/s eta 0:00:01


In [3]:
import time
from os import getenv
import os
from dotenv import load_dotenv
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets
import traitlets
import PIL
from IPython.display import display, Image
from jetcam.utils import bgr8_to_jpeg
import IPython
from jetcam.usb_camera import USBCamera
from facenet_pytorch import MTCNN
from IPython.display import Image
import cv2

class FastMTCNN(object):
    def __init__(self, stride, resize=1, *args, **kwargs):
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        if self.resize != 1:
            frames = [
                cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
                    for f in frames
            ]
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                box = [int(b) for b in box]
                faces.append(frame[box[1]:box[3], box[0]:box[2]])
        
        return faces

In [4]:
device = torch.device('cuda')

fast_mtcnn = FastMTCNN(
    stride=4,
    resize=1,
    margin=14,
    factor=0.6,
    keep_all=True,
    device=device
)

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 2)
model = model.to(device)
model.load_state_dict(torch.load('./model/my_model(1).pth'))
model.eval()
print("model loaded")

load_dotenv()
outframe = None
url = "https://youtu.be/7PYzSXHd6U4"
#livestream = VideoStream()
#linkedstream = LinkedStream(url)
camera = USBCamera(width=224, height=224, capture_fps=30, capture_device=0) # confirm the capture_device number
camera.running = True
print("camera created")

#defaults
active = "Live"
default_radius_size = getenv('DEFAULT_RADIUS')
default_radius_uom = getenv('DEFAULT_RADIUS_UOM')
default_duration = getenv('DEFAULT_DURATION')
default_duration_uom = getenv('DEFAULT_DURATION_UOM')
message = None

#working values
radius_size = getenv('DEFAULT_RADIUS')
radius_uom = getenv('DEFAULT_RADIUS_UOM')
duration = getenv('DEFAULT_DURATION')
duration_uom = getenv('DEFAULT_DURATION_UOM')

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

def change_source(change):
    global active
    active = change['new']

state_widget = ipywidgets.ToggleButtons(options=['Live', 'Playback'], value='Live')
state_widget.observe(change_source, names='value')
image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

def execute(change):
    frame = change['new']
    boxes, probs = fast_mtcnn(frame)
    if boxes is not None:
        print("found face")
        #for box, prob in zip(boxes, probs):
        #    processed = preprocess(frame)
        #    output = model(processed)
        #    output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        #    category_index = output.argmax()
        #    if output[category_index] > 0.6:
        #        color = (0, 0, 255) if category_index else (0, 255, 0)
        #        label = "No MASK" if category_index else "MASK"
        #        print(box)
    #draw_objects(image, counts, objects, peaks)# try this for multiple hand pose prediction
    image_widget.value = bgr8_to_jpeg(frame[:, ::-1, :])

display_handle = display(state_widget, image_widget)
while True:
    execute({'new': camera.value})
    camera.observe(execute, names='value')
    camera.unobserve_all()
    #img = cv2.rectangle(frame, start_point, end_point, color, 2)  # draw class box
    #        text = "{}:{:.2f}".format(label, score[0])
    #        cv2.putText(frame, text, start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)  # print class type with score
    #        frame = cv2.rectangle(frame, start_point, end_point, color, 2)
    #draw.rectangle(box.tolist(), outline=(255,0,0), width=6)

SyntaxError: invalid character in identifier (<ipython-input-4-9699a7c4b965>, line 46)